<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Все-задания-нужно-выполнить-только-с-помощью-sql" data-toc-modified-id="Все-задания-нужно-выполнить-только-с-помощью-sql-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Все задания нужно выполнить только с помощью sql</a></span></li></ul></div>

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
      Address, 
      Country, 
      CustomerName
    FROM Customers
    WHERE Country IN ('Germany','France')
     OR UPPER(CITY) LIKE 'MADRID'
    ''',
    con,
)

,Address,Country,CustomerName
0,Obere Str. 57,Germany,Alfreds Futterkiste
1,Forsterstr. 57,Germany,Blauer See Delikatessen
2,"24, place Kléber",France,Blondel père et fils
3,"C/ Araquil, 67",Spain,Bólido Comidas preparadas
4,"12, rue des Bouchers",France,Bon app'
5,Walserweg 21,Germany,Drachenblut Delikatessend
6,"67, rue des Cinquante Otages",France,Du monde entier
7,"C/ Moralzarzal, 86",Spain,FISSA Fabrica Inter. Salchichas S.A.
8,"184, chaussée de Tournai",France,Folies gourmandes
9,Berliner Platz 43,Germany,Frankenversand


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
      COUNT(*) AS CNT, 
      Country
    FROM Customers
    GROUP BY Country
    ORDER BY CNT DESC
    LIMIT 3
    ''',
    con
)

,CNT,Country
0,13,USA
1,11,Germany
2,11,France


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [9]:
tasks+=1
pd.read_sql(
    '''WITH step_1 AS (
        SELECT 
          OrderDate, 
          ShipperName
        FROM Orders a
        LEFT JOIN Shippers b on a.ShipperID = b.ShipperID
        ORDER BY OrderDate
        LIMIT 10)
    SELECT 
      * 
    FROM step_1
    ORDER BY OrderDate desc
    LIMIT 1
    ''',
    con,
)

,OrderDate,ShipperName
0,02.10.1996,Federal Shipping


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [10]:
tasks+=1
pd.read_sql(
    '''
    SELECT
      p.ProductName, 
      p.Price, 
      od.Quantity, 
      p.Price * od.Quantity as total_amount
    FROM Products AS p
    JOIN OrderDetails AS od ON p.ProductID = od.ProductID
    ORDER BY total_amount DESC
    LIMIT 1
    ''',
    con,
)

,ProductName,Price,Quantity,total_amount
0,Côte de Blaye,263.5,50,13175.0


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [11]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
      p.ProductName, 
      o.OrderID, 
      od.Quantity
    FROM Products p
    JOIN OrderDetails od ON p.ProductID = od.ProductID
    JOIN Orders o ON o.OrderID = od.OrderID
    WHERE p.ProductName like (  SELECT 
                                  p.ProductName
                                FROM Products p
                                JOIN OrderDetails od ON p.ProductID = od.ProductID
                                JOIN Orders o ON o.OrderID = od.OrderID
                                GROUP BY p.ProductName
                                ORDER BY SUM(Quantity) DESC
                                LIMIT 1)
    ''',
    con,
)

,ProductName,OrderID,Quantity
0,Gorgonzola Telino,10253,20
1,Gorgonzola Telino,10272,40
2,Gorgonzola Telino,10273,15
3,Gorgonzola Telino,10325,4
4,Gorgonzola Telino,10335,25
5,Gorgonzola Telino,10342,56
6,Gorgonzola Telino,10356,30
7,Gorgonzola Telino,10359,70
8,Gorgonzola Telino,10363,20
9,Gorgonzola Telino,10374,30


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [12]:
tasks+=1

pd.read_sql(
    '''
    SELECT
      s.SupplierName, 
      s.Country, 
      s.ContactName, 
      s.Phone, 
      o.OrderID
    FROM Products p
    JOIN OrderDetails od ON p.ProductID = od.ProductID
    JOIN Orders o ON o.OrderID = od.OrderID
    JOIN Suppliers s ON p.SupplierID = s.SupplierID
    GROUP BY s.SupplierName
    ORDER BY count(o.OrderID) desc
    LIMIT 5
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone,OrderID
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755,10277
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343,10255
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448,10281
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010,10252
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323,10253


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
      cus.Country, 
      cat.CategoryName, 
      SUM(od.Quantity * p.Price) AS TotalCost
    FROM Products AS p
    JOIN OrderDetails od ON p.ProductID = od.ProductID
    JOIN Orders o ON o.OrderID = od.OrderID
    JOIN Categories cat ON cat.CategoryID = p.CategoryID
    JOIN Customers cus ON cus.CustomerID = o.CustomerID
    WHERE LOWER(Country) LIKE 'brazil'
    ''',
    con,
)

,Country,CategoryName,TotalCost
0,Brazil,Condiments,40215.25


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
      max_.max_qp as max,
      min_.min_qp as min,
      max_.max_qp - min_.min_qp AS "Разница между суммами" 
    FROM   (SELECT 
              Quantity * Price as max_qp
            FROM Products p
            JOIN OrderDetails od ON p.ProductID = od.ProductID
            JOIN Orders o ON o.OrderID = od.OrderID
            JOIN Customers cus ON cus.CustomerID = o.CustomerID
            WHERE Country like 'USA'
            ORDER BY max_qp DESC
            LIMIT 1) max_, 
            (SELECT 
              Quantity * Price  as min_qp
            FROM Products p
            JOIN OrderDetails od ON p.ProductID = od.ProductID
            JOIN Orders o ON o.OrderID = od.OrderID
            JOIN Customers cus ON cus.CustomerID = o.CustomerID
            WHERE Country like 'USA'
            ORDER BY min_qp
            LIMIT 1) min_
    ''',
    con,
)


,max,min,Разница между суммами
0,5270.0,37.5,5232.5


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [15]:
tasks+=1
pd.read_sql(
    '''
    WITH step_1 AS (SELECT distinct 
                      BirthDate,
                      o.EmployeeID,
                      e.FirstName||' '||e.LastName AS Name
                    FROM Orders o
                    JOIN Employees e ON o.EmployeeID = e.EmployeeID
                    ORDER BY SUBSTRING(BirthDate, 7) DESC, SUBSTRING(BirthDate, 4,2) DESC, SUBSTRING(BirthDate, 0,3) DESC
                    LIMIT 3),
         step_2 AS (SELECT 
                      count(o.OrderID) as cnt_,
                      o.EmployeeID
                    FROM Orders o
                    JOIN Employees e ON o.EmployeeID = e.EmployeeID
                    GROUP BY o.EmployeeID
                    ORDER BY SUBSTRING(BirthDate, 7) DESC, SUBSTRING(BirthDate, 4,2) DESC, SUBSTRING(BirthDate, 0,3) DESC
                    limit 3)
    SELECT 
      cnt_, 
      Name 
    FROM step_1 s1
    JOIN step_2 s2 ON s1.EmployeeID = s2.EmployeeID
    ''', 
    con,
)


,cnt_,Name
0,6,Anne Dodsworth
1,29,Nancy Davolio
2,31,Janet Leverling


10. Сколько банок крабового мяса всего было заказано.

In [16]:
tasks+=1
pd.read_sql(
    '''
    SELECT
      p.ProductName, 
      SUM(od.Quantity * SUBSTR(Unit, 1, 2)) AS Tins
    FROM Products p
    JOIN OrderDetails od ON p.ProductID = od.ProductID
    WHERE UPPER(ProductName) LIKE '%MEAT%'
    ''',
    con,
)

,ProductName,Tins
0,Boston Crab Meat,6144


In [17]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
